In [3]:
import pandas as pd
import numpy as np
import datetime as dt
from datetime import date
import os

In [4]:
day_ran = str(date.today())
day_ran

'2025-03-20'

In [6]:
# csv paths
# trip_csv = r"C:\Users\will.davis\OneDrive - Fiore & Sons, Inc\1A Data Analysys\P4 Truck wait times\data\Samsara_Trips.csv"
# new_trip_csv = r"C:\Users\will.davis\OneDrive - Fiore & Sons, Inc\1A Data Analysys\P4 Truck wait times\data\Samara_trips_2025.xlsx"
# trip_data = pd.read_csv(trip_csv)

'C:\\Users\\will.davis\\OneDrive - Fiore & Sons, Inc\\1A Data Analysys\\P4 Truck wait times\\generated data\\samsara_wait_times_2025-03-20.xlsx'

In [36]:
%%time
# # csv paths
# trip_csv = r"C:\Users\will.davis\OneDrive - Fiore & Sons, Inc\1A Data Analysys\P4 Truck wait times\data\Samsara_Trips.csv"
new_trip_csv = r"C:\Users\will.davis\OneDrive - Fiore & Sons, Inc\1A Data Analysys\P4 Truck wait times\data\Samara_trips_2025.csv"
trip_data = pd.read_csv(new_trip_csv)
# trip_data = pd.read_excel(new_trip_csv)
# trip_data = trip_data[trip_data['endAddress_name'] != ""]
trip_data['wait_time'] = ""
# make a list of dsates and one of vins
date_list = trip_data['Date'].unique()
vin_list = trip_data['Vin Number'].unique()
df_list = []
for vin in vin_list:
    vin_df = trip_data[trip_data['Vin Number']==vin]
    for day in date_list:
        date_subset = vin_df[vin_df['Date'] == day]
        date_subset = date_subset.reset_index()
        # make sure that this is still in the right order to calculate
        date_subset = date_subset.sort_values(by='start_time')
        df_length = len(date_subset)
        if df_length > 1:
            for row in range(0,df_length-1):
                current = date_subset.iloc[row]
                next_row = date_subset.iloc[row+1]
                if current['endAddress_name'] == next_row['startAddress_name']:
                    wait_time = pd.to_datetime(next_row['start_time']) - pd.to_datetime(current['end_time'])
                    # date_subset.loc[row, 'wait_time'] = wait_time.total_seconds()
                    date_subset.loc[row, 'wait_time'] = wait_time.total_seconds() / 60 # may be computationally heavy 
            df_list.append(date_subset)


# full_df['wait_time'] = full_df['wait_time'] / 60 # may be computationally heavy 

full_df = pd.concat(df_list)
no_blank_times = full_df[full_df['wait_time'] != ""]
no_blank_times = no_blank_times[no_blank_times['wait_time'] < 180]
no_blank_times = no_blank_times[no_blank_times['wait_time'] > 10]
pivot_table = pd.pivot_table(no_blank_times, values='wait_time', index='endAddress_name', aggfunc='mean')

CPU times: total: 15.8 s
Wall time: 16.5 s


In [32]:
# make path for each day that the data is compiled 
end_file = f'samsara_wait_times_{str(date.today())}.xlsx'
end_path = r"C:\Users\will.davis\OneDrive - Fiore & Sons, Inc\1A Data Analysys\P4 Truck wait times\generated data\{}".format(end_file)
end_path
# create excel for spot checks
writer = pd.ExcelWriter(end_path, engine="xlsxwriter")
no_blank_times.to_excel(writer, sheet_name="full_df")
pivot_table.to_excel(writer, sheet_name="Pivot")
writer.close()

In [31]:
%%time
# ***TRy to speed this operatio***

new_trip_csv = r"C:\Users\will.davis\OneDrive - Fiore & Sons, Inc\1A Data Analysys\P4 Truck wait times\data\Samara_trips_2025.xlsx"
trip_data = pd.read_excel(new_trip_csv)
# trip_data = trip_data[trip_data['endAddress_name'] != ""]
trip_data['wait_time'] = ""
# sort data set 
sorted_set = trip_data.sort_values(by=["Vin Number","Date","start_time"])

df_length = len(sorted_set)
for row in range(0,df_length-1):
    current = sorted_set.iloc[row]
    next_row = sorted_set.iloc[row+1]
    if (current["Vin Number"] == next_row["Vin Number"]) & (current['endAddress_name'] == next_row['startAddress_name']) & (current["Date"] == next_row["Date"]):
        wait_time = pd.to_datetime(next_row['start_time']) - pd.to_datetime(current['end_time'])
        sorted_set.loc[row, 'wait_time'] = wait_time.total_seconds() / 60 # may be computationally heavy 


no_blank_times = sorted_set[sorted_set['wait_time'] != ""]
no_blank_times = no_blank_times[no_blank_times['wait_time'] < 180]
no_blank_times = no_blank_times[no_blank_times['wait_time'] > 10]
pivot_table = pd.pivot_table(no_blank_times, values='wait_time', index='endAddress_name', aggfunc='mean')

#wow this is 5-10 seconds slower lol 

CPU times: total: 54.9 s
Wall time: 56.6 s


In [6]:
full_df.columns

Index(['index', 'CalendarSKey', 'id', 'startMs', 'endMs', 'startLocation',
       'endLocation', 'distanceMeters', 'fuelConsumedMl', 'tollMeters',
       'driverId', 'codriverIds', 'startOdometer', 'endOdometer', 'assetIds',
       'startAddress_id', 'startAddress_name', 'startAddress_address',
       'endAddress_id', 'endAddress_name', 'endAddress_address',
       'startCoordinates_latitude', 'startCoordinates_longitude',
       'endCoordinates_latitude', 'endCoordinates_longitude', 'vehicle_id',
       'start_time', 'end_time', 'Minutes', 'Gallons', 'Date', 'Miles', 'MPG',
       'Hours', 'UT%', 'EquipmentID', 'Vin Number', 'HRRef', 'Tags',
       'wait_time'],
      dtype='object')